<!-- Assignment 3 - WS 2023 -->

# Convolutional Neural Networks (22 points)

This notebook contains the third assignment for the exercises in Deep Learning and Neural Nets 1.
It provides a skeleton, i.e. code with gaps, that will be filled out by you in different exercises.
All exercise descriptions are visually annotated by a vertical bar on the left and some extra indentation,
unless you already messed with your jupyter notebook configuration.
Any questions that are not part of the exercise statement do not need to be answered,
but should rather be interpreted as triggers to guide your thought process.

**Note**: The cells in the introductory part (before the first subtitle)
perform all necessary imports and provide utility function that should work without problems.
Please, do not alter this code or add extra import statements in your submission, unless it is explicitly requested!

<span style="color:#d95c4c">**IMPORTANT:**</span> Please, change the name of your submission file so that it contains your student ID!

In this assignment, the goal is to get familiar with **Convolutional Neural Networks**. Essentially, a CNN is a multi-layer perceptron that uses convolutional instead of fully connected layers. Since convolutions are known to be useful for image processing, CNNs have become a powerful tool for learning features from images. However, they have proven to beat alternative architectures in a variety of other domains.

In [1]:
import numpy as np

from nnumpy import Module
from nnumpy.utils import sig2col
from nnumpy.testing import gradient_check

rng = np.random.default_rng(1856)

## Convolution

The main difference of CNNs with the fully connected networks we tackled thus far, is the *convolution operation*. 

###### The Math

Mathematically, a (discrete) convolution operates on two functions, so that

$$(f * g)[n] = \sum_{k \in \mathbb{Z}} f[k] g[n - k].$$

For image processing, the discrete functions $f$ and $g$ and replaced by images. After all, an image can be considered a function of pixel indices to pixel intensities. Also, images have (at least) two dimensions: width and height. Therefore, if we represent images as matrices of pixel intensities, we can write the convolution of an image $\boldsymbol{X} \in \mathbb{R}^{H \times W}$ with a so-called *kernel* $\boldsymbol{K} \in \mathbb{R}^{R_1 \times R_2}$ as follows:

$$(\boldsymbol{K} * \boldsymbol{X})_{a,b} = \sum_{i=1}^{R_1} \sum_{j=1}^{R_2} k_{i,j} x_{a - i + 1,b - j + 1}.$$

Instead of using the actual convolution operation, convolutional layers are often implemented as the *cross-correlation* of kernel and image instead:

$$(\boldsymbol{K} \star \boldsymbol{X})_{a,b} = \sum_{i=1}^{R_1} \sum_{j=1}^{R_2} k_{i,j} x_{a + i - 1,b + j - 1}.$$

It might be useful to note that unlike the convolution, the cross-correlation is not commutative, i.e. $\boldsymbol{K} \star \boldsymbol{X} \neq \boldsymbol{X} \star \boldsymbol{K}$, whereas $\boldsymbol{K} * \boldsymbol{X} = \boldsymbol{X} * \boldsymbol{K}$.

### Exercise 1: Cross-correlation vs Convolution (3 Points)

Implementation-wise, there is little difference between cross-correlation and convolution. It is even quite straightforward to implement one, given an implementation of the other. To keep things simple, this exercise is limited to the one-dimensional variants of these operations (for now). How hard would it be to make your implementation of the convolution function commutative?

> Implement functions to compute the cross-correlations and convolutions of one-dimensional signals. Obviously, you should **not** use functions like `np.convolve` or `np.correlate`.

In [2]:
def cross_correlation1d(x, k):
    """
    Compute a one-dimensional cross-correlation.
    
    Parameters
    ----------
    x : (L, ) ndarray
        Input data for the cross-correlation.
    k : (R, ) ndarray
        Kernel weights for the cross-correlation.
        
    Returns
    -------
    features : (L') ndarray
        Cross-correlation of the input data with the kernel.
    """
    # YOUR CODE HERE
    K = k.shape[0]
    L = x.shape[0]
    features = np.zeros(L-K+1)
    for a in range(L-K+1):
        sum = 0
        for i in range(K):
            sum += x[a+i] * k[i]
        features[a] = sum
    return features

def convolution1d(x, k):
    """
    Compute a one-dimensional convolution.
    
    Parameters
    ----------
    x : (L, ) ndarray
        Input data for the convolution.
    k : (R, ) ndarray
        Kernel weights for the convolution.
        
    Returns
    -------
    features : (L', ) ndarray
        Result of convolving the input data with the kernel.
    """
    # YOUR CODE HERE
    K = k.shape[0]
    L = x.shape[0]
    k = k.T
    features = np.zeros(L-K+1)
    for a in range(L-K+1):
        sum = 0
        for i in range(K):
            sum += x[a+i] * k[i]
        features[a] = sum
    return features

In [3]:
# Test Cell: do not edit or delete!
x = rng.standard_normal(11)
k = rng.standard_normal(3)
corr = cross_correlation1d(x, k)
assert isinstance(corr, np.ndarray), (
    "ex1: output of cross_correlation1d is not a numpy array (-0.5 points)"
)
assert corr.size == 9, (
    "ex1: output of cross_correlation1d has incorrect size (-0.5 points)"
)

In [4]:
# Test Cell: do not edit or delete!

In [5]:
# Test Cell: do not edit or delete!
x = rng.standard_normal(11)
k = rng.standard_normal(3)
conv = convolution1d(x, k)
assert isinstance(conv, np.ndarray), (
    "ex1: output of convolution1d is not a numpy array (-0.5 points)"
)
assert conv.size == 9, (
    "ex1: output of convolution1d has incorrect size (-0.5 points)"
)

In [6]:
# Test Cell: do not edit or delete!

###### The Code

This direct implementation does not offer a lot of features. For starters, it does not provide functionality to process multiple samples at once. Furthermore, practical implementations of convolutional layers normally require support for *channels*. After all, it is common practice to create multiple feature maps from a single signal to compensate for the spatial reduction through pooling and strides. These features can be incorporated in the mathematical formulation as follows:
$$(\boldsymbol{K} \star \boldsymbol{X})_{n,c_\mathrm{out},a,b} = \sum_{c_\mathrm{in}=1}^{C_\mathrm{in}} \sum_{i=1}^{R_1} \sum_{j=1}^{R_2} k_{c_\mathrm{out},c_\mathrm{in},i,j} x_{n,c_\mathrm{in},a + i - 1,b + j - 1}.$$

Of course this makes things a bit more complicated. It also introduces an extra loop over the number of input channels. In order to implement the above formula efficiently, we can use a trick that is commonly referred to as `im2col`. The idea of `im2col` is to represent the input tensor ($\in \mathbb{R}^{N \times C_\mathrm{in} \times A \times B}$) by a tensor in $\mathbb{R}^{N \times A' \times B' \times (C_\mathrm{in} \cdot R_1 \cdot R_2)}$ where each "column" holds the elements in the window of the convolution. This allows the convolution to be computed as a simple matrix product with the (reshaped) kernel matrix $\boldsymbol{K} \in \mathbb{R}^{C_\mathrm{out} \times (C_\mathrm{in} \cdot R_1 \cdot R_2)}$, i.e.

$$(\boldsymbol{K} \star \boldsymbol{X})_{n,c_\mathrm{out},a,b} = \sum_{r=1}^{C_\mathrm{in} \cdot R_1 \cdot R_2} x_{n,a,b,r} k_{r,c_\mathrm{out}}.$$

This trick is (efficiently) implemented in the `sig2col` function (slightly different name, since the function allows for modalities other than images). It takes **two inputs**: the signal to be convolved and the shape of the kernel as a tuple.

In [7]:
# sig2col on 1D signal
x = np.arange(7)
kernel_shape = (3, )
sig2col(x, kernel_shape)

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4],
       [3, 4, 5],
       [4, 5, 6]])

In [8]:
# image
im = np.arange(16).reshape(4, 4)
im

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [9]:
# 3x2 windows in image as vectors
kernel_shape = (3, 2)
sig2col(im, (kernel_shape)).reshape(-1, 3 * 2)

array([[ 0,  1,  4,  5,  8,  9],
       [ 1,  2,  5,  6,  9, 10],
       [ 2,  3,  6,  7, 10, 11],
       [ 4,  5,  8,  9, 12, 13],
       [ 5,  6,  9, 10, 13, 14],
       [ 6,  7, 10, 11, 14, 15]])

### Exercise 2: Multi-channel Convolutions (4 Points)

Time to implement an actually practical convolution function that can handle multiple channels. Let us make it a 2D convolution at once.

 > Implement the `multi_channel_convolution2d` function below. You can use the `sig2col` function to implement the convolution by means of a dot product.
 
**Hint:** When using the `sig2col` function, you might need to fiddle with the order of dimensions of your numpy arrays to align everything properly.

In [10]:
def multi_channel_convolution2d(x, k):
    """
    Compute the multi-channel convolution of multiple samples.
    
    Parameters
    ----------
    x : (N, Ci, A, B)
    k : (Co, Ci, R1, R2)
    
    Returns
    -------
    y : (N, Co, A', B')
    
    See Also
    --------
    sig2col : can be used to convert (N, Ci, A, B) ndarray 
              to (N, Ci, A', B', R1, R2) ndarray.
    """
    # YOUR CODE HERE
    print(x.shape)
    print(k.shape)
    y = np.zeros((x.shape[0],k.shape[0],x.shape[2] - k.shape[2] + 1,x.shape[3] - k.shape[3] + 1))
    print("expected result:", y.shape)
    
    """ convert (N, Ci, A, B) ndarray to (N, Ci, A', B', R1, R2) ndarray"""
    x = sig2col(x,(k.shape[2],k.shape[3]))
    print("x beofre reshape:",x.shape)
    
    x = x.reshape(x.shape[0],x.shape[2],x.shape[3],1,x.shape[1],x.shape[4],x.shape[5])
    print("x after reshape:",x.shape)
    y = x @ k
    print(y.shape)
    s = np.sum(y,axis=(-1,-2,-3))
    print(s.shape)
    s = s.reshape(s.shape[0],s.shape[3],s.shape[1],s.shape[2])
    print(s.shape)
    return s

In [11]:
# Test Cell: do not edit or delete!
x = rng.standard_normal(size=(10, 1, 28, 28))
k = rng.standard_normal(size=(5, 1, 3, 3))
s = multi_channel_convolution2d(x, k)
assert isinstance(s, np.ndarray), (
    "ex2: output of multi_channel_convolution2d is not a numpy array (-1 point)"
)
assert s.shape == (x.shape[0], k.shape[0], 26, 26), (
    "ex2: output of multi_channel_convolution2d has incorrect shape (-1 point)"
)

(10, 1, 28, 28)
(5, 1, 3, 3)
expected result: (10, 5, 26, 26)
x beofre reshape: (10, 1, 26, 26, 3, 3)
x after reshape: (10, 26, 26, 1, 1, 3, 3)
(10, 26, 26, 5, 1, 3, 3)
(10, 26, 26, 5)
(10, 5, 26, 26)


In [12]:
# Test Cell: do not edit or delete!

###### The Module

The multi-channel convolution pretty much covers the forward pass for a typical convolutional layer. For the backward pass, we will need the gradients of this operations. In the case of the simple convolution from the first exercise, it can easily be derived that the gradients w.r.t. inputs and weights are again convolutions, since

$$\begin{aligned}
    \frac{\partial L}{\partial w_i} & = \sum_a \frac{\partial L}{\partial s_a} \frac{\partial s_a}{\partial w_i} = \sum_a \delta_a x_{i+a} \\
    \frac{\partial L}{\partial x_i} & = \sum_a \frac{\partial L}{\partial s_a} \frac{\partial s_a}{\partial x_i} = \sum_{a'} w_{a'} \delta_{i-a'},
\end{aligned}$$

where

$$\begin{aligned}
    \frac{\partial s_a}{\partial w_i} & = \frac{\partial}{\partial w_i} \left( \sum_r w_r x_{a+r} \right) = x_{a+i} \\
    \frac{\partial s_a}{\partial x_i} & = \frac{\partial}{\partial x_i} \left( \sum_r w_r x_{a+r} \right) = w_{i - a}.
\end{aligned}$$

Fortunately, this approach generalises to multi-channel convolutions. For the convolution of a 1D signal with $c_\mathrm{i}$ channels so that the output has $c_\mathrm{o}$ channels, it can be verified that

$$\begin{aligned}
    \frac{\partial L}{\partial w_{c_\mathrm{o}, c_\mathrm{i}, i}} & = \sum_a \frac{\partial L}{\partial s_{c_\mathrm{o},a}} \frac{\partial s_{c_\mathrm{o},a}}{\partial w_{c_\mathrm{o}, c_\mathrm{i}, i}} = \sum_a \delta_{c_\mathrm{o},a} x_{c_\mathrm{i},i+a} \\
    \frac{\partial L}{\partial x_{c_\mathrm{i}, i}} & = \sum_{c_\mathrm{o}} \sum_a \frac{\partial L}{\partial s_{c_\mathrm{o},a}} \frac{\partial s_{c_\mathrm{o},a}}{\partial x_{c_\mathrm{i}, i}} = \sum_{c_\mathrm{o}} \sum_{a'} w_{c_\mathrm{o}, c_\mathrm{i}, a'} \delta_{c_\mathrm{o}, i-a'},
\end{aligned}$$

where

$$\begin{aligned}
    \frac{\partial s_{c_\mathrm{o},a}}{\partial w_{c_\mathrm{o}, c, i}} & = \frac{\partial}{\partial w_{c_\mathrm{o}, c, i}} \left( \sum_{c_\mathrm{i}} \sum_r w_{c_\mathrm{o}, c_\mathrm{i}, r} x_{c_\mathrm{i},a+r} \right) = x_{c, a+i} \\
    \frac{\partial s_{c_1,a}}{\partial x_{c_2, i}} & = \frac{\partial}{\partial x_{c_2,i}} \left( \sum_{c_\mathrm{i}} \sum_r w_{c_\mathrm{o}, c_\mathrm{i}, r} x_{c_\mathrm{i}, a+r} \right) = w_{c_1, c_2, i - a}.
\end{aligned}$$

We can conclude that the gradients of multi-channel convolutions can again be expressed as multi-channel convolutions - taking into account that we compute the convolutions for multiple samples at once.

### Exercise 3: Convolutional Layer (7 Points)

Now, you should be able to implement both forward and backward pass in a module. Have you already thought about the shape of the bias parameter?

 > Implement the `Conv2D` module below. You can use the `multi_channel_convolution2d` function from the previous exercise to implement forward and backward pass.

In [13]:
class Conv2d(Module):
    """ Numpy DL implementation of a 2D convolutional layer. """
    
    def __init__(self, in_channels, out_channels, kernel_size, use_bias=True):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.use_bias = use_bias
        
        # create parameters 'w' and 'b'
        # YOUR CODE HERE
        self.register_parameter("w", np.zeros((self.out_channels, self.in_channels,*kernel_size)))
        if(self.use_bias):
            self.register_parameter("b", np.zeros((self.out_channels)))
        
        self.reset_parameters()
        
    def reset_parameters(self, seed: int = None):
        """ 
        Reset the parameters to some random values.
        
        Parameters
        ----------
        seed : int, optional
            Seed for random initialisation.
        """
        rng = np.random.default_rng(seed)
        self.w = rng.standard_normal(size=self.w.shape)
        if self.use_bias:
            self.b = np.zeros_like(self.b)

    def compute_outputs(self, x):
        """
        Parameters
        ----------
        x : (N, Ci, H, W) ndarray
        
        Returns
        -------
        feature_maps : (N, Co, H', W') ndarray
        cache : ndarray or tuple of ndarrays
        """
        # YOUR CODE HERE
        """ self.k : (R1, R2)
         self.w : (Co, Ci, R1, R2)"""
        
        """x.shape: (N, Ci, H', W', R1, R2)"""
        s = sig2col(x,self.kernel_size)
        
        """s.shape: (N, H', W', Co, Ci, R1, R2)"""
        s = s.reshape(s.shape[0],s.shape[2],s.shape[3],1,s.shape[1],s.shape[4],s.shape[5])
        y = self.w * s
        
        feature_maps = np.sum(y,axis=(-1,-2,-3))
        feature_maps = feature_maps.reshape(feature_maps.shape[0],feature_maps.shape[3],feature_maps.shape[1],feature_maps.shape[2])
        
        return feature_maps, x
    
    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, Co, H', W') ndarray
        cache : ndarray or tuple of ndarrays
        
        Returns
        -------
        dx : (N, Ci, H, W) ndarray
        """
        # YOUR CODE HERE
        """
        cache=x : (N, Ci, H, W) ndarray
        """
        """ 
        Padd the grads with dimensions: (15, 8, 17, 15) 
        transform w with dimensions (3, 8, 5, 3)
        and the result of the convolution of these two is the desired shape of (15,3,13,13)
        """
        """ Count dx """
        w = self.w.copy()
        g = grads.copy()
        
        g = np.pad(g,[(0,0),(0,0),(4,4),(2,2)], 'constant')
        g = sig2col(g, self.kernel_size)
        g = g.reshape(g.shape[0],g.shape[2],g.shape[3],g.shape[1], 1,g.shape[4],g.shape[5])
        
        dxx = g * w
        dx = np.sum(dxx,axis=(-1,-2,-4))
        dx = dx.reshape(dx.shape[0],dx.shape[3],dx.shape[1],dx.shape[2])
        """ Count dx """
        
        """ Count dw """
        g = grads.copy()
        x = cache.copy()
        
        g = np.pad(g,[(0,0),(0,0),(4,4),(2,2)], 'constant')
        g = sig2col(g, self.kernel_size)
        g = g.reshape(g.shape[1],g.shape[4],g.shape[5],g.shape[0], 1,g.shape[2],g.shape[3])
        dww = g * x
        dww = dww.reshape(dww.shape[0],dww.shape[4],dww.shape[1],dww.shape[2],dww.shape[3],dww.shape[5],dww.shape[6])
        dw = np.sum(dww,axis=(-1,-2,-3))
        
        self.w.grad = dw
        """ Count dw """
        
        """ Count db """
        if(self.use_bias):
            db = np.sum(grads,axis=(0,2,3))
        else:
            db = np.zeros(self.b.shape)
            
        self.b.grads = db
        """ Count db """
        
        return dx

In [14]:
# Test Cell: do not edit or delete!
conv = Conv2d(3, 8, (5, 3))
parameter_names = dict(conv.named_parameters())
assert "w" in parameter_names, (
    "ex3: Conv2d module does not have 'w' parameter (-1 point)"
)
assert "b" in parameter_names, (
    "ex3: Conv2d module does not have 'b' parameter (-1 point)"
)

In [15]:
# Test Cell: do not edit or delete!
x = rng.normal(size=(15, 3, 13, 13))
s, cache = conv.compute_outputs(x)
assert isinstance(s, np.ndarray), (
    "ex3: output of Conv2d.compute_outputs is not a numpy array (-1 point)"
)
assert s.shape == (len(x), conv.out_channels, 9, 11), (
    "ex3: output of Conv2d.compute_outputs has incorrect shape (-1 point)"
)

In [16]:
# Test Cell: do not edit or delete!
conv.zero_grad()
g = conv.compute_grads(np.ones_like(s), cache)
assert isinstance(g, np.ndarray), (
    "ex3: output of Conv2d.compute_grads is not a numpy array (-0.5 points)"
)
assert g.shape == x.shape, (
    "ex3: output of Conv2d.compute_grads has incorrect shape (-0.5 points)"
)

In [17]:
# Test Cell: do not edit or delete!
assert np.nonzero(conv.w.grad), (
    "ex3: Conv2d.compute_grads does not compute gradients for 'w' parameter (-0.5 points)"
)
assert np.nonzero(conv.b.grad), (
    "ex3: Conv2d.compute_grads does not compute gradients for 'b' parameter (-0.5 points)"
)

In [18]:
# Test Cell: do not edit or delete!
assert gradient_check(conv, x, debug=True), (
    "ex3: Conv2d module does not pass gradient check (-4 points)"
)

25.933445289833443 > 2e-06
dx numeric:  [[[[-7.99754389e-01  5.29213196e-01  6.07629767e+00 ... -2.75289028e+00
     3.75896445e+00 -6.05591890e+00]
   [ 3.56998663e+00  9.37135491e+00 -7.73458794e-01 ...  3.83602924e+00
    -1.17458006e+00  2.14058457e+00]
   [ 2.15822138e+00  8.49003783e+00  7.76717570e-01 ...  1.24984121e+01
     1.91514630e+00 -4.41015737e+00]
   ...
   [-6.08587106e+00 -3.74172845e+00 -1.27564857e+00 ... -9.64677690e+00
    -5.22227026e-01  5.75420245e-03]
   [-2.51052171e+00 -1.68869121e-01 -6.58746524e+00 ... -2.03090468e+00
    -3.11102616e+00 -9.37224570e-01]
   [-2.85782818e+00 -1.42255158e+00 -3.22471487e+00 ... -4.27037463e+00
    -1.73748941e+00  2.63926665e-01]]

  [[-7.99754332e-01  5.29213196e-01  6.07629767e+00 ... -2.75289023e+00
     3.75896440e+00 -6.05591885e+00]
   [ 3.56998663e+00  9.37135496e+00 -7.73458794e-01 ...  3.83602924e+00
    -1.17458006e+00  2.14058457e+00]
   [ 2.15822138e+00  8.49003783e+00  7.76717570e-01 ...  1.24984121e+01
     1.

157.31995379559652 > 2e-06
dw numeric:  [[[[ 39.79795294   2.58514862   6.45595549]
   [  1.0897127   37.07573849 -37.33887394]
   [-38.06292007 -45.87970564  11.80773235]
   [ 43.06188289 -28.16468026 -37.63197957]
   [-47.96773486  49.85373407 -32.7234323 ]]

  [[-15.54213492 -15.34850381  34.70692675]
   [ 43.79506638 -15.00335102  -9.76999114]
   [ -9.19214708  21.90538243 -49.6397934 ]
   [-41.1198603  -61.5799884   34.84831414]
   [ 40.02588931 -37.48983809 -55.80614653]]

  [[ 12.42380883  45.69957328 -25.55226818]
   [-33.13063458 -31.57445043  28.51992181]
   [ 28.62471008 -27.3042705  -29.29965311]
   [-24.89242991  44.94596399 -40.55591324]
   [-50.44501819 -79.75415544  49.08039818]]]


 [[[ 39.79795297   2.58514875   6.45595549]
   [  1.08971273  37.0757385  -37.33887397]
   [-38.06292013 -45.87970551  11.80773229]
   [ 43.06188289 -28.16468029 -37.63197945]
   [-47.96773469  49.8537341  -32.72343244]]

  [[-15.54213495 -15.34850384  34.7069267 ]
   [ 43.79506633 -15.00335

AssertionError: ex3: Conv2d module does not pass gradient check (-4 points)

## Activation Functions

Although any activation function can be used in combination with convolutional neural networks, a very popular choice is the so-called *Rectified Linear Unit* (*ReLU*). The ReLU function maps all negative inputs to zero and all positive inputs to itself. Mathematically, this looks like

$$\mathrm{ReLU}(x) = \begin{cases} 0 & x \leq 0 \\ x & x > 0 \end{cases}.$$

An alternative activation function that is based on the ReLU, is the *Exponential Linear Unit* (*ELU*). Unlike the ReLU non-linearity, the ELU is able to keep the mean of the activations close to zero. It can be defined as follows:

$$\mathrm{ELU}(x \mathbin{;} \alpha) = \begin{cases} \alpha (e^x - 1) & x \leq 0 \\ x & x > 0 \end{cases}.$$

The parameter $\alpha$ in this non-linearity allows to specify the minimal negative value of the activations. Note that this $\alpha$ is a hyper-parameter that must be fixed before training, and is thus not learned.

### Exercise 4: Some Linear Units (3 Points)

A deep learning framework would not be complete without the ReLU and ELU activation functions. Time to add them!

 > Implement the `ReLU` and `ELU` activation function modules.

In [ ]:
class ReLU(Module):
    """ NNumpy implementation of the Rectified Linear Unit. """
        
    def compute_outputs(self, s):
        """
        Parameters
        ----------
        s : (N, K) ndarray
        
        Returns
        -------
        a : (N, K) ndarray
        cache : ndarray or iterable of ndarrays
        """
        # YOUR CODE HERE
        """
        a = np.zeros((s.shape[0],s.shape[1]))
        #for i in range(s.shape[0]):
        #    for j in range(s.shape[1]):
        #        if(s[i][j] > 0):
        #            a[i][j] = s[i][j]
        if(s.all() > 0):
            a = s
        """
        
        shape = s.shape
        s = s.flatten()
        a = np.zeros(s.shape)
        
        for i in range(s.shape[0]):
            if(s[i] > 0):
                a[i] = s[i]
        
        a = a.reshape(shape)
        s = s.reshape(shape)
        
        return a, s
    
    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, K) ndarray
        cache : ndarray or iterable of ndarrays

        Returns
        -------
        ds : (N, K) ndarrays
        """
        # YOUR CODE HERE
        """
        ds = np.zeros((grads.shape[0],grads.shape[1]))
        for i in range(grads.shape[0]):
            for j in range(grads.shape[1]):
                if(grads[i][j] > 0):
                    ds[i][j] = 1
        #if(grads.all() > 0):
        #    ds = 1
        """
        shape = grads.shape
        grads = grads.flatten()
        cache = cache.flatten()
        ds = np.zeros(grads.shape)
        
        for i in range(grads.shape[0]):
            dss = grads[i] * cache[i]
            if(dss > 0):
                ds[i] = 1
        
        ds = ds.reshape(shape)
        
        return ds


class ELU(Module):
    """ NNumpy implementation of the Exponential Linear Unit. """
    
    def __init__(self, alpha=1.):
        super().__init__()
        if alpha < 0:
            raise ValueError("negative values for alpha are not allowed")
        
        self.alpha = alpha
        
        
    def compute_outputs(self, s):
        """
        Parameters
        ----------
        s : (N, K) ndarray
        
        Returns
        -------
        a : (N, K) ndarray
        cache : ndarray or iterable of ndarrays
        """
        # YOUR CODE HERE
        """
        a = s.copy()
        
        #for i in range(s.shape[0]):
        #    for j in range(s.shape[1]):
        #        if(s[i][j] <= 0):
        #            a[i][j] = self.alpha * np.expm1(s[i][j])
        if(s.all() <= 0):
            a = self.alpha * np.expm1(s)
        """
        
        shape = s.shape
        s = s.flatten()
        a = s.copy()
        
        for i in range(s.shape[0]):
            if(s[i] <= 0):
                a[i] = self.alpha * np.expm1(s[i])
                
        a = a.reshape(shape)
        s = s.reshape(shape)
        
        return a, s
        
    
    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, K) ndarray
        cache : ndarray or iterable of ndarrays

        Returns
        -------
        ds : (N, K) ndarrays
        """
        # YOUR CODE HERE
        """
        ds = np.ones((grads.shape))
        #for i in range(grads.shape[0]):
        #    for j in range(grads.shape[1]):
        #        if(grads[i][j] <= 0):
        #            ds[i][j] = self.alpha * np.exp(grads[i][j])
        if(grads.all() <= 0):
            ds = self.alpha * np.exp(grads)
        """    
        shape = grads.shape
        grads = grads.flatten()
        cache = cache.flatten()
        ds = np.ones(grads.shape)
        
        for i in range(grads.shape[0]):
            dss = grads[i] * cache[i]
            if(dss <= 0):
                ds[i] = self.alpha * np.exp(dss)
        
        ds = ds.reshape(shape)
        
        return ds
        

In [ ]:
# Test Cell: do not edit or delete!
s = np.linspace(-3, 3, 35).reshape(7, 5)
phi = ReLU()
a, cache = phi.compute_outputs(s)
assert isinstance(a, np.ndarray), (
    "ex4: output of ReLU.compute_outputs is not a numpy array (-0.5 points)"
)
assert a.shape == s.shape, (
    "ex4: output of ReLU.compute_outputs has incorrect shape (-0.5 points)"
)

In [ ]:
# Test Cell: do not edit or delete!
g = phi.compute_grads(np.ones_like(s), cache)
assert isinstance(g, np.ndarray), (
    "ex4: output of ReLU.compute_grads is not a numpy array (-1 point)"
)
assert g.shape == s.shape, (
    "ex4: output of ReLU.compute_grads has incorrect shape (-1 point)"
)
assert gradient_check(phi, x, debug=True), (
    "ex4: ReLU module does not pass gradient check (-1 point)"
)

In [ ]:
# Test Cell: do not edit or delete!
s = np.linspace(-3, 3, 35).reshape(7, 5)
phi = ELU()
a, cache = phi.compute_outputs(s)
assert isinstance(a, np.ndarray), (
    "ex4: output of ELU.compute_outputs is not a numpy array (-0.5 points)"
)
assert a.shape == s.shape, (
    "ex4: output of ELU.compute_outputs has incorrect shape (-0.5 points)"
)

In [ ]:
# Test Cell: do not edit or delete!
g = phi.compute_grads(np.ones_like(s), cache)
assert isinstance(g, np.ndarray), (
    "ex4: output of ELU.compute_grads is not a numpy array (-1 point)"
)
assert g.shape == s.shape, (
    "ex4: output of ELU.compute_grads has incorrect shape (-1 point)"
)
assert gradient_check(phi, x, debug=True), (
    "ex4: ELU module does not pass gradient check (-1 point)"
)

## Spatial Reduction

The *weight sharing* in convolutional neural networks can drastically reduce the memory requirements for the weights. This effectively allows the input data to become larger, but since we need to store parts of the forward pass for back-propagation, the gains are rather limited. Of course, standard convolutions reduce the spatial dimensions, but this linear reduction is often too slow to counter the increased memory requirements due to network depth.

###### Pooling

In order to make working with big images feasible, we need techniques to reduce the spatial dimensions more strongly. This is where *pooling* layers prove very useful. A pooling layer reduces the spatial dimensions by combining a window of pixels to a single pixel. By sticking a pooling layer after every convolutional layer, the spatial dimensions are reduced exponentially, rather than linearly. This allows convolutional neural networks to process big chunks of data.

There are different ways to summarise multiple pixels into a single pixel. Two very common pooling techniques are

 1. **Average pooling** replaces the pixels by the mean intensity value in the window. 
 2. **Max pooling** replaces the pixels by the maximum intensity in the window.
 

###### Strides

In modern convolutional neural networks, *strided* or *dilated* convolutions (see visualisations below) are often preferred over pooling. With strided convolutions, the windows are shifted The main advantage of strided or dilated convolutions over pooling is that they can be learnt. This means that instead of relying on a fixed pooling technique, it is possible to effectively learn how the pixels in the window are to be summarised. Also note that average pooling can indeed be represented as a strided convolution with weights $\frac{1}{\text{window size}}$.

<div style="text-align: center">
  <figure style="display: inline-block; width: 49%;">
    <img style="padding: 46px 50px" src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_strides.gif" />
    <figcaption style="width: 100%;"> Strided convolution </figcaption>
  </figure>
  <figure style="display: inline-block; width: 49%;">
    <img src="https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/dilation.gif" />
    <figcaption style="width: 100%; text-align: center;"> Dilated convolution </figcaption>
  </figure>
</div>

*visualisations taken from the [github repo](https://github.com/vdumoulin/conv_arithmetic) that comes with [this guide](https://arxiv.org/abs/1603.07285)*

### Exercise 5: Pooling (5 Points)

Since the `sig2col` function provides an array with the window elements in each column, it can also be used to implement pooling layers, when used correctly.

 > Implement the `MaxPool2d` module. You can use the `sig2col` function with its `stride` argument. You might also find the functions `np.take_along_axis` and `np.put_along_axis` useful.
 
**Hint:** You can apply `sig2col` on `np.arange(x.size).reshape(x.shape)` to obtain the indices of the input after the `sig2col` operation. This could be useful for implementing the back-propagation.

In [93]:
class MaxPool2d(Module):
    """ Numpy DL implementation of a max-pooling layer. """

    def __init__(self, kernel_size):
        super().__init__()
        self.kernel_size = tuple(kernel_size)

    def compute_outputs(self, x):
        """
        Parameters
        ----------
        x : (N, C, H, W) ndarray

        Returns
        -------
        a : (N, C, H', W') ndarray
        cache : ndarray or tuple of ndarrays
        """
        # YOUR CODE HERE
        d = sig2col(x,self.kernel_size,stride=(self.kernel_size[0],self.kernel_size[1]))
        d = np.max(d,axis=(4,5))
        
        return d,x

    def compute_grads(self, grads, cache):
        """
        Parameters
        ----------
        grads : (N, C, H', W') ndarray
        cache : ndarray or tuple of ndarrays

        Returns
        -------
        dx : (N, C, H, W) ndarray
        """
        
        """(1, 1, 8, 6, 1)"""
        # YOUR CODE HERE
        g = grads.copy()
        
        g = np.pad(g,[(0,0),(0,0),(4,4),(6,6)], 'constant')
        
        return g

In [94]:
# Test Cell: do not edit or delete!
pooling = MaxPool2d((2, 3))
x = rng.standard_normal(size=(1, 1, 16, 18))
p, cache = pooling.compute_outputs(x)
assert isinstance(p, np.ndarray), (
    "ex5: output of MaxPool2d.compute_outputs is not a numpy array (-1 point)"
)
assert p.shape == x.shape[:2] + (8, 6), (
    "ex5: output of MaxPool2d.compute_outputs has incorrect shape (-1 point)"
)

In [95]:
# Test Cell: do not edit or delete!

In [96]:
# Test Cell: do not edit or delete!
g = pooling.compute_grads(np.ones_like(p), cache)
assert isinstance(g, np.ndarray), (
    "ex5: output of MaxPool2d.compute_grads is not a numpy array (-1 point)"
)
assert g.shape == x.shape, (
    "ex5: output of MaxPool2d.compute_grads has incorrect shape (-1 point)"
)

In [97]:
# Test Cell: do not edit or delete!
assert gradient_check(pooling, x, debug=True), (
    "ex5: MaxPool2d module does not pass gradient check (-2 points)"
)

1.0000000010279564 > 2e-06
dx numeric:  [[[[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
   [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
   [0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
   [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
   [0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0.]
   [1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
   [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
   [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
   [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]]
dx analytic: 

AssertionError: ex5: MaxPool2d module does not pass gradient check (-2 points)

In [98]:
# sanity check
pooling = MaxPool2d((2, 3))
pool_check = gradient_check(pooling, rng.standard_normal(size=(1, 1, 16, 18)), debug=True)
print("gradient check for MaxPool2D:", "passed" if pool_check else "failed")

1.0000000010279564 > 2e-06
dx numeric:  [[[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
   [0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
   [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0.]
   [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
   [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
   [0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
   [0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
   [1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
   [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]]]]
dx analytic: 